<a href="https://colab.research.google.com/github/Untick/InspectrumClinic_RecSys/blob/main/Paul%20Himyak/save_modified_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!wget -qc "https://storage.yandexcloud.net/terratraineeship/23_InspectrumClinic_RS/datasets/%D0%B4%D0%B0%D1%82%D0%B0%D1%81%D0%B5%D1%82%20%D0%BF%D1%80%D0%BE%D1%84%D0%BF%D0%B0%D1%82%D0%BE%D0%BB%D0%BE%D0%B3%20%D1%87%D0%B0%D1%81%D1%82%D1%8C.xlsx"


In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import uuid
import re
import numpy as np
import seaborn as sns
from datetime import datetime

In [16]:
from google.colab import drive

# Монтируем Google Диск
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
df = pd.read_excel("/content/датасет профпатолог часть.xlsx")


In [18]:
FIELD_CLIENT_AGE = "КлиентВозраст"
FIELD_CLIENT_BIRTH_DAY = "КлиентДатаРождения"
FIELD_CLIENT_GENDER = "КлиентПол"
FIELD_CLIENT_GENDER_M = 0
FIELD_CLIENT_GENDER_F = 1
FIELD_CONCLUSION = "ЗаключениеМК"
FIELD_CONCLUSION_FIT = 1
FIELD_CONCLUSION_PART_FIT = 2
FIELD_CONCLUSION_NOT_FIT = 0

FIELD_DISPANSARY_GROUP = "ДиспансернаяГруппа"
FIELD_DISPANSARY_GROUP_I = 0
FIELD_DISPANSARY_GROUP_II = 1
FIELD_DISPANSARY_GROUP_III = 2
FIELD_DISPANSARY_GROUP_OTHER = 3

FIELD_HARMFUL_FACTORS = "ВредныеФакторы"


FIELD_NEW_AGE_CATEGORY = "ВозрастКатегория"
FIELD_NEW_MKBS = "mkbs"

FIELD_CLIENT_PROFESSION = "Профессия"
FIELD_NEW_CLIENT_PROFESSION = "Profession"
FIELD_NEW_CLIENT_ADDITIONS = "Дополнения сводные"

In [19]:
mkb_columns = df.filter(regex="(?i)МКБ10").columns

In [20]:
additions_columns = df.filter(regex="(?i)Дополнение_Значение$").columns




In [30]:
diag_columns = df.filter(regex="(?i)Диагноз\s?Представление").columns

In [31]:
df[diag_columns]

,ПсихиатрияНаркология1_ДиагнозПредставление1,Офтальмология1_ДиагнозПредставление1,Оториноларингология1_ДиагнозПредставление1,ХирургНеИсп1_ДиагнозПредставление1,Гинекология1_ДиагнозПредставление1,Стоматология1_ДиагнозПредставление1,Дерматовенерология1_ДиагнозПредставление1,Рентгенология1_ДиагнозПредставление1,Неврология1_ДиагнозПредставление1,Терапия1_ДиагнозПредставление1,...,Психиатрия1_ДиагнозПредставление2,Психиатрия2_ДиагнозПредставление1,Психиатрия2_ДиагнозПредставление2,Рентгенология1_ДиагнозПредставление2,Рентгенология2_ДиагнозПредставление2,Психиатрия3_ДиагнозПредставление1,Психиатрия3_ДиагнозПредставление2,Психиатрия4_ДиагнозПредставление1,Психиатрия4_ДиагнозПредставление2,Оториноларингология3_ДиагнозПредставление1
0,Общий медицинский осмотр,Общий медицинский осмотр,Общий медицинский осмотр,Общий медицинский осмотр,Общий медицинский осмотр,Общий медицинский осмотр,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Общий медицинский осмотр,Общий медицинский осмотр,NaN,NaN,NaN,NaN,Общий медицинский осмотр,Общий медицинский осмотр,Общий медицинский осмотр,Гипертензивная [гипертоническая] болезнь с пре...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Общий медицинский осмотр,Миопия,NaN,NaN,NaN,NaN,Общий медицинский осмотр,NaN,NaN,Общий медицинский осмотр,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Общий медицинский осмотр,Общий медицинский осмотр,Общий медицинский осмотр,NaN,Общий медицинский осмотр,NaN,NaN,Общий медицинский осмотр,Общий медицинский осмотр,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Астигматизм,Общий медицинский осмотр,NaN,NaN,Общий медицинский осмотр,NaN,NaN,Общий медицинский осмотр,Общий медицинский осмотр,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1979,Общий медицинский осмотр,Общий медицинский осмотр,Общий медицинский осмотр,NaN,NaN,Кариес дентина,Общий медицинский осмотр,NaN,Общий медицинский осмотр,Общий медицинский осмотр,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980,Общий медицинский осмотр,NaN,Общий медицинский осмотр,NaN,Общий медицинский осмотр,NaN,Общий медицинский осмотр,NaN,Общий медицинский осмотр,Общий медицинский осмотр,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
base_columns = [FIELD_CLIENT_BIRTH_DAY, FIELD_CLIENT_GENDER, FIELD_CONCLUSION, FIELD_DISPANSARY_GROUP, FIELD_HARMFUL_FACTORS, FIELD_CLIENT_PROFESSION]

In [33]:
general_columns = np.concatenate((base_columns, mkb_columns, additions_columns, diag_columns))

In [34]:
df[general_columns]

,КлиентДатаРождения,КлиентПол,ЗаключениеМК,ДиспансернаяГруппа,ВредныеФакторы,Профессия,ПсихиатрияНаркология1_МКБ101,Офтальмология1_МКБ101,Оториноларингология1_МКБ101,ХирургНеИсп1_МКБ101,...,Психиатрия1_ДиагнозПредставление2,Психиатрия2_ДиагнозПредставление1,Психиатрия2_ДиагнозПредставление2,Рентгенология1_ДиагнозПредставление2,Рентгенология2_ДиагнозПредставление2,Психиатрия3_ДиагнозПредставление1,Психиатрия3_ДиагнозПредставление2,Психиатрия4_ДиагнозПредставление1,Психиатрия4_ДиагнозПредставление2,Оториноларингология3_ДиагнозПредставление1
0,1976-10-24,Женский,Годен,I,11.4,Ведущий специалист по организации обуения,Z00.0,Z00.0,Z00.0,Z00.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1987-11-18,Мужской,Годен,IIIа,4.2.5,Ведущий специалист,Z00.0,Z00.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1985-09-08,Мужской,ГоденСКоррекциейЗрения,II,4.2.5,Главный менеджер,Z00.0,H52.1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1988-04-25,Мужской,Годен,I,11.1,Главный специалист,NaN,Z00.0,Z00.0,Z00.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1984-04-17,Мужской,ГоденСКоррекциейЗрения,II,11.1,Старший специалист,NaN,H52.2,Z00.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1977,1972-11-12,Мужской,NaN,NaN,6.1,Директор по строительству,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1978,1986-07-09,Мужской,NaN,NaN,6.1,Начальник производственно-технического отдела,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1979,1994-05-12,Мужской,NaN,NaN,"5.1,23",Помощник кладовщика,Z00.0,Z00.0,Z00.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980,1972-04-17,Женский,NaN,NaN,23,Повар,Z00.0,NaN,Z00.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
df = df[general_columns]

In [36]:
#Удаляем случайные вбросы
df.drop([843, 851, 1022, 1024], inplace = True)

#Убираем пустые колонки
empty_columns = df.columns[df.isna().all()]
df.drop(empty_columns, inplace = True, axis = 1)

In [37]:
df[general_columns].to_excel("/content/drive/MyDrive/profpatolog_modified.xlsx")